In [ ]:
# # Will iterate over all graphs on nn vertices from nn = start_n to nn = stop_n - 1.
# # Can use shortcut_edges = edcount to start at edcount edges instead of starting with the complete graph on start_n vertices;
# # this is useful if the kernel gets interrupted while working on graphs on 10 vertices since it takes a few days to process
# # all 11 million+ graphs on 10 vertices.

# # These parameters can be set here or can be set in a command line instance and then run the notebook from the command line.
# start_n = 2
# stop_n = 10
# shortcut_edges = False


# # Do you want to remove previously calculated data?
# # remove_old_data = True
# remove_old_data = False

In [ ]:
## Name of directory in which to save the data files.
path_prefix = 'data_v2'

# Removes the results of previous calculations
if remove_old_data == True:
    import shutil
    
    try:
        shutil.rmtree(path_prefix)
    except FileNotFoundError:
        pass

In [ ]:
# load the read/write functions
# Eventually replace this local load to loading from the public github repo
load('spectator_floor_number_read_write_functions.py')
load('spectator_floor_functions.py')

In [ ]:
%%cython # use cython to speed up computations -- This jupyter magic call cannot be in a .py script file

# use garbage collection to manage memory usage
import gc
gc.enable()


def usp_comp(amat):
    """ Return the spectator number of the graph, which is defined to be
    the size of the smallest unique-shortest-path complement of a graph.
    
    :param amat: A graph or adjacency matrix of a graph.
    """
    # Accept graph or matrix input
    try:
        # Get size from the number of columns, if amat is a matrix
        nn = amat.ncols()
    except AttributeError:
        # If amat is a graph or graph6_string, convert to adjacency matrix
        amat = amat.adjacency_matrix()
        # Get size from number of colummns
        nn = amat.ncols()
    if not amat:
        return nn - 1
    
    # Use fact that (i,j)-entry of A^k is the number of i-j walks of length k
    # to determine the length of the longest unique shortest path (the largest k
    # for which there exists an (i,j)-entry of A^k equal to 1.)
    A = amat + 2
    AA = A + 0
    compsize = nn - 1
    while min(min([yy for yy in xx if yy]) for xx in AA) == 1:
        compsize -= 1
        AA = AA*A
    
    return compsize



def Glabel(G):
    """
    Returns the graph6_string of the canonical labeling of graph G using the sage algorithm
    to determine the canonical labeling.
    
    :param G: A graph object.
    """
    return G.canonical_label(algorithm='sage').graph6_string()


def edgeclasses(G):
    """
    Generator function to generate the automorphism groups of the edges in graph G.
    
    :param G: A graph object.
    """
    Aut = G.automorphism_group()
    needs = {(xx[0], xx[1]): True for xx in G.edges()}
    while needs:
        anedge = next(iter(needs))
        yield anedge
        for xx in Aut.orbit(anedge, action='OnPairs'):
            if (xx[0], xx[1]) in needs:
                del needs[(xx[0], xx[1])]
            if (xx[1], xx[0]) in needs:
                del needs[(xx[1], xx[0])]

                
def deletions(G):
    """
    Generator function to generate the minors of G which are achievable by deleting a single edge
    from G.
    Returns the graph6_string of the canonical labeling.
    
    :param G: A graph object.
    """
    for ed in edgeclasses(G):
        H = G.copy()
        H.delete_edge(ed)
        yield Glabel(H)

        
def contractions(G):
    """
    Generator function to generate the minors of G which are achievable by either contracting a single
    edge in G or deleting a single isolated vertex in G.
    Returns the graph6_string of the canonical labeling.
    
    :param G: A graph object.
    """
    for ed in edgeclasses(G):
        H = G.copy()
        H.contract_edge(ed)
        yield Glabel(H)
    # Also covers isolated vertex deletion
    if 0 in G.degree():
        H = G.copy()
        H.delete_vertex(G.degree().index(0))
        yield Glabel(H)

In [ ]:
import os

# Check to see if the data directory is set up; otherwise, create the data directory
if path_prefix not in os.listdir('.'):
    os.mkdir(path_prefix)
if 'minimals_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/minimals_dict')
if 'uspcm_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/uspcm_dict')
if 'full_uspcm_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/full_uspcm_dict')
if 'seen_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/seen_dict')
if 'completed_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/completed_dict')

    
# Initialize the uspcm_dict: The dictionary that starts with the spectator numbers and ends with containing
# the spectator minor floor numbers for the graphs.
# Initialize minimals_dict: The dictionary of graphs which are minimal with respect to the spectator minor floor number.
# Initialize seen_dict: The dictionary of graphs that have been processed by spec_floor_first_pass().
# Initialize completed_dict: The dictionary of graphs that have been processed by second_pass().
# All of these are nested dictionaries. uspcm_dict, seen_dict, and completed_dict have a first layer of keys of the form
# {nn}_verts and second layer of keys of the form {ee}_edges. For example, the set of graphs on 4 vertices and 5 edges 
# that have been processed by second_pass() is accessable via completed_dict['4_verts']['5_edges']
# The set of graphs which are spectator minor floor minimal for spectator number 3 is accessable via minimals_dict['3_spectators']
uspcm_dict, minimals_dict, seen_dict, completed_dict = get_spectator_number_dictionaries(path_prefix)

# The code depends on the partial_uspcm_dict for the graph on 0 vertices and for the graph on 1 vertex existing.
starter_uspcm_dict = {'0_verts':{}, '1_verts':{}}
starter_uspcm_dict['0_verts']['0_edges'] = {Glabel(Graph(0)): 0}
starter_uspcm_dict['1_verts']['0_edges'] = {Glabel(Graph(1)): 0}
write_partial_uspcm_dict(0, 0, starter_uspcm_dict, path_prefix)
write_partial_uspcm_dict(1, 0, starter_uspcm_dict, path_prefix)


# Iterate over all graphs on nn vertices from nn = start_n to nn = stop_n - 1.
for nn in range(start_n, stop_n):
    print(f'Working on graphs with {nn} vertices.')

    # Start with the complete graph on nn vertices, work down to the empty graph on nn vertices
    edcount = Integer((nn*(nn-1))/2)
    K_n = Graph(nn).complement()
    # Add to the uspcm_dict
    uspcm_dict[f'{nn}_verts'][f'{K_n.num_edges()}_edges'] = {Glabel(K_n): nn - 2}
    
    if shortcut_edges:   
        # Used if having to restart partway through graphs on 10 vertices since those 11million+ graphs take multiple days to process.
        edcount = shortcut_edges
    
    # First pass: set values, giving every superminor a chance.
    while edcount:
        # Working on graphs on nn vertices with decreasing number of edges down to 0 edges
        
        spec_floor()
        
        # Save progress
        write_partial_uspcm_dict(nn, edcount, uspcm_dict, path_prefix)
        if edcount > 1:
            write_partial_uspcm_dict(nn, edcount-1, uspcm_dict, path_prefix)
        write_partial_seen_dict(nn, edcount, seen_dict, path_prefix)


        
        
        # Second pass: check for minimality
        minimals_dict, completed_dict = determine_minimals(nn, edcount, minimals_dict, uspcm_dict, completed_dict, save=True, path_prefix=path_prefix)

        write_partial_completed_dict(nn, edcount, completed_dict, path_prefix)
        
        edcount -= 1


print('Done.')